In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [3]:
class Audio_to_Text:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        self.model_id = "openai/whisper-large-v3"

        self.model = AutoModelForSpeechSeq2Seq.from_pretrained(self.model_id)
        self.processor = AutoProcessor.from_pretrained(self.model_id)
        self.model.to(self.device)

        self.pipe = pipeline(
            "automatic-speech-recognition",
            model=self.model,
            tokenizer=self.processor.tokenizer,
            feature_extractor=self.processor.feature_extractor,
            torch_dtype=self.torch_dtype,
            device=self.device
        )

    def run(self, audio_file):
        return self.pipe(audio_file, generate_kwargs={"language": "arabic"})["text"]

In [4]:
import torch
from transformers import MarianMTModel, MarianTokenizer, M2M100ForConditionalGeneration, M2M100Tokenizer # Added missing imports

In [5]:
class Translator:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
        self.tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
        self.tokenizer.src_lang = "ar"
        self.model.to(self.device)

    def run(self, arabic_text):
        inputs = self.tokenizer(arabic_text, return_tensors="pt").to(self.device)  # Move inputs to the correct device
        generated_tokens = self.model.generate(**inputs, forced_bos_token_id=self.tokenizer.get_lang_id("en"))
        translated_text = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        return translated_text

In [6]:
from huggingface_hub import login
import os
login()
access_token = os.environ.get("HUGGING_FACE_HUB_TOKEN")

In [7]:
!pip install diffusers

In [8]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

In [9]:
class Text_to_Image:
  def __init__(self):
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.model_id = "CompVis/stable-diffusion-v1-4"

    self.scheduler = EulerDiscreteScheduler.from_pretrained(self.model_id, subfolder="scheduler")
    self.pipe = StableDiffusionPipeline.from_pretrained(self.model_id, scheduler=self.scheduler, torch_dtype=torch.float16)
    self.pipe = self.pipe.to(self.device)

  def run(self, prompt):
    image = self.pipe(prompt).images[0]
    return image


In [10]:
class VoiceToImage:
  def __init__(self):
    self.audio_to_text = Audio_to_Text()
    self.translator = Translator()
    self.text_to_image = Text_to_Image()

  def run(self, audio):
    arabic_text = self.audio_to_text.run(audio)
    english_text = self.translator.run(arabic_text)
    image = self.text_to_image.run(english_text[0])
    return image

In [11]:
voice_to_image = VoiceToImage()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
!pip install gradio

In [ ]:
import gradio as gr

def record_audio(audio):
    image = voice_to_image.run(audio)
    return image

# Create the Gradio interface
iface = gr.Interface(
    fn=record_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=gr.Image(),
    title="Audio To Image",
    description="Record your audio and generate image."
)

# Launch the interface
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fd2dc8fdd26244839a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=arabic, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=arabic.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]